# Spellchecker

Spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [2]:
lines = open("holbrook.txt").readlines()
data = []
# Write your code here
for line in lines:
    indices = []
    str1 = []
    str2 = []
    line = line.split()
    for i in range(len(line)):
        if "|" in line[i]:
            indices.append(i)
            str1.append(line[i].split("|")[0])
            str2.append(line[i].split("|")[1])
        else:
            str1.append(line[i])
            str2.append(line[i])
    data.append({'original': str1, 'corrected': str2, 'indexes': indices})

assert(data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [3]:
test = data[:100]
train = data[100:]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [4]:
from collections import Counter

def unigram(word):
    # Write your code here.
    corrected_lines = []
    for line in train:
        corrected_lines.extend(line['corrected'])
        for w in corrected_lines:
          w.lower()
    cnt = Counter(corrected_lines)
    return cnt[word]
    

def prob(word):
    count = unigram(word)
    corrected_lines = []
    for line in train:
        corrected_lines.extend(line['corrected'])
    total = len(corrected_lines)
    prob = count/total
    return prob

# Test your code with the following
assert(unigram("me")==87)

## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [5]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [6]:
def get_candidates(token):
  tokens = []
  for t in train:
        tokens.extend(t["corrected"])
        #tokens.extend(t['original'])
  token_set = set(tokens)
  dmin = 100000
  minDs = []
  for t in token_set:
      d = edit_distance(token, t)
      if d == dmin:
          minDs.append(t)
      if d < dmin:
          dmin = d
          minDs = [t]
  minDs.sort(reverse=True)
  return minDs
        
# Test your code as follows
assert get_candidates("minde") == ['mine', 'mind']

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [8]:
def correct(sentence):
    tokens = []
    for t in train:
        tokens.extend(t["corrected"])
    token_set = set(tokens)
    for word in sentence:
        if word not in token_set:
            candidates = get_candidates(word)
            highest_prob = 0
            for candidate in candidates:
                probability = prob(word)
                if probability >= highest_prob:
                    highest_prob = probability
                    sentence = [candidate if (x == word) else x for x in sentence]
    return sentence

assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [ ]:
def accuracy(test):
    incorrect_count = 0
    correct_count = 0
    for sentence in test:
      my_correction = correct(sentence['original'])
      for i in range(len(my_correction)):
        if my_correction[i] != sentence['corrected'][i]:
          incorrect_count += 1
          #print(my_correction[i])
          #print(sentence['corrected'][i])
        else:
          correct_count +=1
    total = correct_count + incorrect_count
    accuracy = correct_count/total
    return accuracy

print(accuracy(test))
# slow to run but returns 0.8148804251550045

## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


Some modifications which could be implemented to improve the accuracy of the algorithm are to increase the unigram probability to check to bigrams or trigrams. Increased values of n in n-gram probabilty checks take into consideration the context in which a token exists. Unigram probability ignores the liklihood of any words coexistance and focuses soley on the existance of individual words in the text. It is recommended that different values on n should be tested to see which comes back with the highest accuracy for the text data in question. As many of these sentences are relatively short then it is unlikely that a high value of n would give significant accuracy increase but bigrams and trigrams could be worth exploring for a text like this.

Another modification which could lead to accuracy improvement would be to consider lemmatization. Lemmatization is similar to stemming. It removes endings in text to return the base of a word known as its lemma. iTs implementation should improve accuracy by stopping words which share the same basic meaning being corrected to other words. For example if "begin" exists in our training set, and "beginning" does not, then we could say that the word beginning should not be changed to begin as it is a word with a similar meaning to begin.

A modification which could improve accuracy and be useful alongside the lemmatization modification would be to check that a word which our algorithm does not recognise already exists as a valid word. This could be achieved by checking a dictionary for its existance. It could reduce the effects over lemmatization which can occur in the case where a spelling mistake (word which does not exist) could share the same stem as a correct word in the training data. "Begining" and "Begning" could share the same stem "beg" but should not be ignored as begning is not a valid word. The English wordnet.words() lexical database of semantic relations of words could be incorporated into an implementation of this modification.

Named entity recognition could be used to improve accuracy by recognise names of entities such as "Uncle Ben". This could be achieved through the use of part of speech tagging in sentences. This could impolment the use of trees as studied in lectures. In the implementation of NER case should not be ignored and recognised named entities from the training data should be stored in a list for comparison.

Through studying the results of the accuracy test in the previous task, it became clear that there were some cases where discrepancies between the given holbrook.txt correction and the correct() implementation correction. These discrepancies included:
- numbers (eg. "8" -> "48")
- days of the week ("Tuesday" -> "Thursday")
- case (eg. "No" -> "no")
These common discrepancies specific to the nature of the given text can be removed or reduced through application of some of the approaches discussed below.

Some common discrepancies were seen in number correction where test numbers which did not appear in the train data were corrected to numbers which did appear in the train data. In order to imporve the accuracy of the algorithm in this area there are two possible approaches that could be used:
1. If any number token is detected do not correct that token
2. Add all numbers to the train dataset
It would be more logical to implement the first option as it is more efficient and lends itself to simpler understanding.

A list of days of the week and months of the year could be incorportaed to make up the base of the list of named entities. This modification would be likely to reduce the occurance of incorrect "corrections" such as "Thursday" to "Tuesday" where Tuesday exists in the train set but Thursday does not exist there in the original example.

Another modification which could be emplyed to increase accuracy in the model would be to convert words which do not appear in the named entity list to lower case. This is an example of normalisation. An issue realted to this modifcation could arise if this were a grammar checker since capital letters are an important feature of English natural language grammar. However, as this algorithm is primarily concerned with the spelling of words ie. letter order: then capitalization can be ignored and remove discrepancies eg. "Please" corrected by algorithm to "please" which reduces accuracy score even though they share a common spelling.



In [ ]:
def get_candidates2(token):
  tokens = []
  for t in train:
        tokens.extend(t["corrected"])
        #tokens.extend(t['original'])
  token_set = set(tokens)
  dmin = 100000
  minDs = []
  for t in token_set:
      d = edit_distance(token, t)
      if d == dmin:
          minDs.append(t)
      if d < dmin:
          dmin = d
          minDs = [t]
  minDs.sort(reverse=True)
  return minDs
        
# Test your code as follows
#assert get_candidates2("minde") == ['mine', 'mind']

def correct2(sentence):
    NER = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    tokens = []
    for t in train:
        tokens.extend(t["corrected"])
    token_set = set(tokens)
    for word in sentence:
        if word not in NER:
          if word not in token_set:
              candidates = get_candidates2(word)
              highest_prob = 0
              for candidate in candidates:
                  probability = prob(word)
                  if probability >= highest_prob:
                      highest_prob = probability
                      sentence = [candidate if (x == word) else x for x in sentence]
    return sentence

assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   


## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [ ]:
def accuracy2(test):
    incorrect_count = 0
    correct_count = 0
    for sentence in test:
      my_correction = correct2(sentence['original'])
      for i in range(len(my_correction)):
        if my_correction[i] != sentence['corrected'][i]:
          incorrect_count += 1
          #print(my_correction[i])
          #print(sentence['corrected'][i])
        else:
          correct_count +=1
    total = correct_count + incorrect_count
    accuracy = correct_count/total
    return accuracy

print(accuracy(test))